In [5]:
from binance.client import Client
from datetime import datetime
import numpy as np
import pandas as pd
import ta

class BinanceAPI:
    """Binance 거래소에 조회 요청하기 위한 api
        
        
    """
    def __init__(self):
        print("This is Biance API")
        
    def api_connect(self):
        """api key 연동
        
            Returns : api key와 secret key를 통해 조회요청한 Client 객체
        
        """
        return Client('ng0Zhq6ea42X3QxMV2RAubZxs508gguTISRwM13lQFFPrDDTxRiqmq3pBvIcvJMy',
                      'vHZDWuQvPf4mcaDvzxwRbtIWDbWuCyFyyG59bCZeTW6A6sd98qbHfCsFDVDdN3wn')
    
    def get_all_coin_list(self):
        """해당 거래소에 상장된 전체 코인 목록을 조회
        
            Returns : 코인 목록을 list 형태로 반환
            
        """
        client = self.api_connect()
        coin_all_list = client.get_all_tickers()
        coin_list = []
        for coin in coin_all_list:
            coin_list.append(coin['symbol'])
        return coin_list
        
    def get_klines(self,coin_name='BTCUSDT', interval='1d', term='6m' ):
        """분봉조회 하는 함수
        
            Args:
                    coin_name : 코인 종류 ( 비트코인<->USD, BTCUSTD)
                    
                    interval : 조회 주기( 1분봉, 15분봉,1일봉 등)
                                    분-M, 시간-h, 일-d, 주-w
                                    ex>1분 : '15M'
                    
                    term: 조회 기간 ( 최근 1개월, 6개월, 1년 등)
                                월-m, 년-y
                                ex>6개월 : '6m'
                    
            Returns:
                    open, close, high, low, volume 이 포함되고 timestamp로 인덱스된 DataFrame
                    
        
        """
        # binance 서버와 통신하기위해 api key 셋팅
        client = self.api_connect()
        
        #조회 주기 및 기간 설정을 위한 변수
        request_interval = Client.KLINE_INTERVAL_1DAY
        request_term = "6 month ago UTC"
        
        #조회 주기 설정
        if interval == '1M':
            request_interval = Client.KLINE_INTERVAL_1MINUTE
        elif interval == '15M':
            request_inerval = Client.KLINE_INTERVAL_15MINUTE
        elif interval == '1h':
            request_inerval = Client.KLINE_INTERVAL_1HOUR
        elif interval == '6h':
            request_inerval = Client.KLINE_INTERVAL_6HOUR
        elif interval == '12h':
            request_inerval = Client.KLINE_INTERVAL_6HOUR
        elif interval == '1d':
            request_inerval = Client.KLINE_INTERVAL_1DAY
        elif interval == '1w':
            request_inerval = Client.KLINE_INTERVAL_1WEEK
        else:
            request_interval = Client.KLINE_INTERVAL_1DAY
        
        #조회 기간 설정
        if term =='1d':
            request_term = "1 day ago UTC"
        elif term == '1w':
            request_term = "1 week ago UTC"
        elif term == '1m':
            request_term = "1 month ago UTC"
        elif term =='6m':
            request_term = "6 month ago UTC"
        elif term =='1y':
            request_term = "1 year ago UTC"
        else:
            request_term = "6 month ago UTC"
        
        # 설정한 값으로 조회
        klines = client.get_historical_klines(coin_name, request_interval, request_term)
        
        #데이터 추출을 위한 변수
        unix_timestamps=[]
        timestamps = []
        closed_prices = []
        open_prices = []
        high_prices= []
        low_prices= []
        volume = []
        
        #시간 추출 unix time
        for kline in klines:
            unix_timestamps.append(kline[0])
        
        #시간 변환  datetime
        for unix_timestamp in unix_timestamps:
            timestamps.append(datetime.fromtimestamp(unix_timestamp/1000))

        # open,close,high,low,volume 추출
        for kline in klines:
            open_prices.append(float(kline[1]))
            high_prices.append(float(kline[2]))
            low_prices.append(float(kline[3]))
            closed_prices.append(float(kline[4]))
            volume .append(float(kline[5]))
        
        #dataframe으로 묶음
        df = pd.DataFrame(list(zip(open_prices,closed_prices,high_prices,low_prices,volume)), index=timestamps, columns = ['open','close','high','low','volume'])
    
        #반환
        return df

In [2]:
binance = BinanceAPI()

This is Biance API


In [3]:
binance.get_klines?

In [4]:
binance.get_klines(coin_name='BTCUSDT', interval='1M',term='1w')

,open,close,high,low,volume
2020-02-19 14:10:00,10096.88,10096.04,10098.04,10095.00,19.261319
2020-02-19 14:11:00,10096.04,10104.83,10107.09,10096.04,18.813159
2020-02-19 14:12:00,10104.83,10109.99,10110.00,10104.70,7.304755
2020-02-19 14:13:00,10109.82,10106.98,10111.08,10102.83,17.475026
2020-02-19 14:14:00,10107.10,10101.96,10108.05,10100.71,24.204904
...,...,...,...,...,...
2020-02-26 14:05:00,9171.90,9163.44,9173.23,9163.35,27.931412
2020-02-26 14:06:00,9163.78,9162.82,9168.43,9162.78,18.945998
2020-02-26 14:07:00,9163.04,9155.75,9164.55,9155.75,50.626405
2020-02-26 14:08:00,9157.11,9152.00,9159.99,9151.17,45.480341


In [7]:
%time binance = BinanceAPI()

This is Biance API
Wall time: 0 ns


In [10]:
%time binance.get_klines(coin_name='BTCUSDT', interval='1M',term='1m')

Wall time: 35.4 s


,open,close,high,low,volume
2020-01-26 14:14:00,8379.01,8376.51,8379.76,8376.11,9.865676
2020-01-26 14:15:00,8375.79,8383.34,8384.99,8375.09,8.772051
2020-01-26 14:16:00,8383.40,8377.39,8386.20,8377.03,17.909003
2020-01-26 14:17:00,8377.39,8386.57,8387.30,8377.39,8.628895
2020-01-26 14:18:00,8386.53,8374.98,8386.53,8373.39,11.562527
...,...,...,...,...,...
2020-02-26 14:10:00,9152.87,9157.74,9160.00,9150.05,47.886885
2020-02-26 14:11:00,9158.34,9162.26,9165.11,9157.21,35.360374
2020-02-26 14:12:00,9162.29,9158.29,9166.58,9157.17,42.683562
2020-02-26 14:13:00,9158.02,9161.22,9165.00,9157.05,43.101786


In [12]:
df=Out[10]

In [13]:
df

,open,close,high,low,volume
2020-01-26 14:14:00,8379.01,8376.51,8379.76,8376.11,9.865676
2020-01-26 14:15:00,8375.79,8383.34,8384.99,8375.09,8.772051
2020-01-26 14:16:00,8383.40,8377.39,8386.20,8377.03,17.909003
2020-01-26 14:17:00,8377.39,8386.57,8387.30,8377.39,8.628895
2020-01-26 14:18:00,8386.53,8374.98,8386.53,8373.39,11.562527
...,...,...,...,...,...
2020-02-26 14:10:00,9152.87,9157.74,9160.00,9150.05,47.886885
2020-02-26 14:11:00,9158.34,9162.26,9165.11,9157.21,35.360374
2020-02-26 14:12:00,9162.29,9158.29,9166.58,9157.17,42.683562
2020-02-26 14:13:00,9158.02,9161.22,9165.00,9157.05,43.101786
